In [ ]:
import functools
import json
import math

import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

# Generate Training and Test Data

The true function that we would like to fit using our neural network is a superposition of 5 sine waves of different frequencies. The resulting function $f_\text{true}$ is a smooth odd function. Our training data has added Gaussian noise $y = f_\text{true}(x) + \epsilon,\, \epsilon \sim \mathcal{N}(0, \sigma^2)$. The test data is noise-free.

In [ ]:
# Define the true function

np.random.seed(0)
F_TRUE_PARAMS = np.random.rand(5) * 2.0 - 1.0


def f_true(x):
    m = F_TRUE_PARAMS.shape[0]
    return (np.sin(x[:, None] * np.arange(m)[None, :] * math.pi) * F_TRUE_PARAMS).sum(axis=-1)

In [ ]:
plt.figure()
plt.plot(np.linspace(-1, 1, 200), f_true(np.linspace(-1, 1, 200)))
plt.title("groundtruth function")
plt.xlabel("x")
plt.ylabel("y")
plt.show()

In [ ]:
# Generate noisy training data
np.random.seed(10)
TRAIN_N = 1500
TRAIN_SIGMA = 0.1
train_x = np.random.uniform(low=-1.0, high=1.0, size=TRAIN_N)
train_y = f_true(train_x) + np.random.randn(TRAIN_N) * TRAIN_SIGMA

plt.figure()
plt.scatter(train_x, train_y, s=3)
plt.title("training data")
plt.xlabel("x")
plt.ylabel("y")
plt.show()

In [ ]:
# Generate noiseless test data
np.random.seed(10)
TEST_N = 500
test_x = np.linspace(-1, 1, TEST_N)
test_y = f_true(test_x)

## Simple Neural Network and Visualization

This section is a simplified version of what you have done in the homework 1.
We train a 1-hidden layer neural network using Stochastic Gradient Descent with Momentum.

In [ ]:
def train_model(model, n_steps, batch_size, lr, seed, train_x=train_x, train_y=train_y):
    np.random.seed(seed)
    torch.manual_seed(seed)
    log_interval = 10
    eval_interval = int(TRAIN_N / batch_size) + 1

    ####################################################################
    # TODO: YOUR CODE HERE
    optimizer = 
    ####################################################################
    ####################################################################
    
    criterion = nn.MSELoss()
    all_indices = []
    while len(all_indices) < n_steps * batch_size:
        all_indices.append(np.random.permutation(TRAIN_N))
    all_indices = np.concatenate(all_indices)
    for step in range(n_steps):
        indices = all_indices[step * batch_size: step * batch_size + batch_size]
        batch_x = torch.from_numpy(train_x[indices]).float().unsqueeze(-1)
        batch_y = torch.from_numpy(train_y[indices]).float()
        pred = model(batch_x).view(-1)
        loss = criterion(batch_y, pred)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if (step + 1) % log_interval == 0:
            print(json.dumps({"step": step, "train_loss": loss.item()}))
        if (step + 1) % eval_interval == 0 or step + 1 == n_steps:
            with torch.no_grad():
                batch_x = torch.from_numpy(test_x).float().unsqueeze(-1)
                batch_y = torch.from_numpy(test_y).float()
                pred = model(batch_x).view(-1)
                loss = criterion(batch_y, pred)
                print(json.dumps({"step": step, "test_loss": loss.item()}))
    return model, batch_x.view(-1).numpy(), pred.numpy()

In [ ]:
# Train a neural network with 1 hidden layer

width = 20
lr = 0.1
batch_size = 32
n_steps = int(TRAIN_N / batch_size * 10)

np.random.seed(100)
torch.manual_seed(100)

model = nn.Sequential(
    nn.Linear(1, width),
    nn.ReLU(),
    nn.Linear(width, 1)
)

model, evaluated_x, evaluated_pred = train_model(model, n_steps, batch_size, lr, 150)

## Let's plot the learned function

plt.figure()
plt.plot(test_x, f_true(test_x), label="truth")
plt.scatter(evaluated_x, evaluated_pred, s=3, c='orange', label="predicted")
plt.legend()
plt.show()

The neural network's training on the data resulted in a learned function, however, its performance is not optimal due to the limited representational power of a single layer. To overcome this, the next section will train a deeper neural network for improved fitting of the true function. But before that, let's first visualize this model using local linearization. Here is the first-order Talor expansion of the learned function $f$ w.r.t. the neural network parameters $\mathbf{w}$:

$$f(x,\mathbf{w}') \approx f(x,\mathbf{w}) + \langle \nabla_\mathbf{w} f(x, \mathbf{w}) |_\mathbf{w} , \mathbf{w}' - \mathbf{w}\rangle$$

Let's first visualize $\nabla_\mathbf{w} f(\cdot, \mathbf{w}) |_\mathbf{w}$.
For each single $x$,
$\nabla_\mathbf{w} f(x, \mathbf{w}) |_\mathbf{w}$ is a vector in $\mathbb{R}^m$,
where $m$ is the number of parameters of the neural network.
So given $n$ datapoints in the training dataset,
we stack these $m$-dim vectors together,
and get a matrix with size $n$ by $m$:

$$
\begin{pmatrix}f(x_1,\mathbf{w}') \\ \vdots \\ f(x_n,\mathbf{w}')\end{pmatrix} =
\begin{pmatrix}f(x_1,\mathbf{w}) \\ \vdots \\ f(x_n,\mathbf{w})\end{pmatrix} +
\begin{pmatrix}\nabla_\mathbf{w} f(x_1, \mathbf{w}) |_\mathbf{w}^T \\ \vdots \\ \nabla_\mathbf{w} f(x_n, \mathbf{w}) |_\mathbf{w}^T\end{pmatrix}(\mathbf{w}' - \mathbf{w})
$$


After that,
we can decompose this matrix with SVD:

$$
\begin{pmatrix}f(x_1,\mathbf{w}') \\ \vdots \\ f(x_n,\mathbf{w}')\end{pmatrix} =
\begin{pmatrix}f(x_1,\mathbf{w}) \\ \vdots \\ f(x_n,\mathbf{w})\end{pmatrix} +
\mathbf{U\Sigma V}^T(\mathbf{w}' - \mathbf{w})
$$

Notice that $\mathbf{V}$ is an orthogonal matrix.

Let's imagine that we have a generalized linear model with a feature matrix corresponding to the linearized features corresponding to each learnable parameter. The singular value and principal features of this matrix is important to us. Here is a visualization of singular values and principal features (rows of $\mathbf{U\Sigma})$:

In [ ]:
def svd_local_linearization(model):
    all_grads = []
    for idx in range(TRAIN_N):
        batch_x = torch.from_numpy(train_x[idx: idx + 1]).float().unsqueeze(-1)
        pred = model(batch_x).view(-1)
        model.zero_grad()
        pred.backward()
        flattened_grads = []
        for param in model.parameters():
            flattened_grads.append(param.grad.view(-1).data.numpy())
        flattened_grads = np.concatenate(flattened_grads)
        all_grads.append(flattened_grads)
    all_grads = np.stack(all_grads)
    u, s, vh = np.linalg.svd(all_grads, full_matrices=False)
    principal_feature = u * s
    return all_grads.shape[1], s, principal_feature


def visualize(m, singular_values, principal_feature):
    fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(14, 4))
    ax1.bar(np.arange(min(len(singular_values), 20)) + 1, singular_values[:20])
    ax1.set_title("Singular Values (Top-20)")

    for idx in range(min(m, 10)):
        ax2.scatter(train_x, principal_feature[:, idx], s=1,
                    label=("feature " + str(idx + 1) if idx < 2 else None))
    ax2.set_title("Principal Features (Top-10)")
    ax2.set_xlabel("x", labelpad=0)
    ax2.set_ylabel("principal feature", labelpad=0)
    ax2.legend()

    im = ax3.scatter(principal_feature[:, 0], principal_feature[:, 1], c=train_x, s=2)
    ax3.set_title("Top-2 Features (x indicated by color)")
    ax3.set_xlabel("feature 1", labelpad=0)
    ax3.set_ylabel("feature 2", labelpad=0)
    plt.colorbar(im, ax=ax3)

    fig.tight_layout(pad=4)
    plt.show()

In [ ]:
m, singular_values, principal_feature = svd_local_linearization(model)
visualize(m, singular_values, principal_feature)

## Training a Deeper Neural Network

In this section, we will train a deeper neural network with different weight scales for initialization. First, **define a fully-connected neural network with 4 hidden layers** and one ReLU activation after each layer except the last one, using the `nn.Sequential` API in PyTorch. Make sure that your model definition passes the assertions below.

In [ ]:
# Define a neural network with 4 hidden layers
# and ReLU activation after each layer except the output layer

width = 20

model = nn.Sequential(
    ####################################################################
    # TODO: YOUR CODE HERE
    ####################################################################
    ####################################################################
)

assert len(model) == 9
assert sum(p.numel() for p in model.parameters()) == 1321
assert list(model(torch.randn(10, 1)).shape) == [10, 1]

### When the weight scale is too small...

The training of a deep neural network is highly impacted by the initialization of its parameters. Let's see what will happen if we initialize each entry in the weight matrices with random values drawn uniformly from the interval `[-0.03, 0.03]`.

In [ ]:
# Initialize the neural network

def naive_init(scale, module):
    if isinstance(module, nn.Linear):
        nn.init.uniform_(module.weight, -scale, scale)
        nn.init.zeros_(module.bias)


np.random.seed(200)
torch.manual_seed(200)
model.apply(functools.partial(naive_init, 0.03))

**Complete the code below to calculate and display the L2-norm of the gradients for each weight matrix**, based on a provided batch of training data.

In [ ]:
def print_gnorms(model):
    batch_x = torch.from_numpy(train_x).float().unsqueeze(-1)
    batch_y = torch.from_numpy(train_y).float()
    pred = model(batch_x).view(-1)
    criterion = nn.MSELoss()
    loss = criterion(batch_y, pred)
    model.zero_grad()
    loss.backward()

    for name, param in model.named_parameters():
        if name.endswith(".weight"):
            with torch.no_grad():
                ####################################################################
                # TODO: YOUR CODE HERE
                ####################################################################
                ####################################################################
                print(name, "{:.8f}".format(gnorm))


print_gnorms(model)

**Question: What are the gradient norms of each layer?** Copy the output of the last cell to your submission of the written assignment with your descriptions.

Here is the visualization of principal features and singular values of local linearization of the neural network *before* the neural network is trained:

In [ ]:
m, singular_values, principal_feature = svd_local_linearization(model)
visualize(m, singular_values, principal_feature)

If you find these figures difficult to read, printing out the values of the principal feature matrix may help you understand what happened:

In [ ]:
principal_feature

Then let's try to train this model.

In [ ]:
lr = 0.1
batch_size = 32
n_steps = int(TRAIN_N / batch_size * 10)

model, evaluated_x, evaluated_pred = train_model(model, n_steps, batch_size, lr, 250)

plt.figure()
plt.plot(test_x, f_true(test_x), label="truth")
plt.scatter(evaluated_x, evaluated_pred, s=3, c='orange', label="predicted")
plt.legend()
plt.show()

**Question: Describe the performance of this model.** Please include the answer in your submission of the written assignment.

Then let's visualize singular values and principal features of this model's local linearization after it has already been trained:

In [ ]:
m, singular_values, principal_feature = svd_local_linearization(model)
visualize(m, singular_values, principal_feature)

In [ ]:
principal_feature

**Question: Describe your observation of the principal features and singular values before and after training.** Please include the figures and your answer in your submission of the written assignment.

### When the weight scale is too large...

Let's see what will happen if we instead initialize each entry in the weight matrices with random values drawn uniformly from the interval `[-3.0, 3.0]`.

In [ ]:
np.random.seed(300)
torch.manual_seed(300)
model.apply(functools.partial(naive_init, 3.0))

In [ ]:
print_gnorms(model)

**Question: What are the gradient norms of each layer?** Copy the output of the last cell to your submission of the written assignment with your descriptions.

Let's do some visualization before training:

In [ ]:
m, singular_values, principal_feature = svd_local_linearization(model)
visualize(m, singular_values, principal_feature)

In [ ]:
lr = 0.1
batch_size = 32
n_steps = int(TRAIN_N / batch_size * 10)

model, evaluated_x, evaluated_pred = train_model(model, n_steps, batch_size, lr, 350)

plt.figure()
plt.plot(test_x, f_true(test_x), label="truth")
plt.scatter(evaluated_x, evaluated_pred, s=3, c='orange', label="predicted")
plt.legend()
plt.show()

**Question: What happened when we try to train this model?** Please include the answer in your submission of the written assignment.

### Better initialization methods

**Implement a better initialization method** based on what you have learned on this course.

HINT: a `module` with type `nn.Linear` has attributes `in_features` and `out_features`.

In [ ]:
def better_init(module):
    if isinstance(module, nn.Linear):
        ########################################################################
        # TODO: YOUR CODE HERE
        ########################################################################
        ########################################################################
        nn.init.zeros_(module.bias)


np.random.seed(600)
torch.manual_seed(600)
model.apply(better_init)

In [ ]:
print_gnorms(model)

**Question: What are the gradient norms of each layer?** Copy the output of the last cell to your submission of the written assignment with your descriptions.

Let's visualize this freshly initialized model.

In [ ]:
m, singular_values, principal_feature = svd_local_linearization(model)
visualize(m, singular_values, principal_feature)

**Question: Compare what you have seen in these figures and compare it with the visualization of the model initialized with very large weight scales before trianing.** Please include the figures and your answer in your submission of the written assignment.

In [ ]:
lr = 0.1
batch_size = 32
n_steps = int(TRAIN_N / batch_size * 10)

model, evaluated_x, evaluated_pred = train_model(model, n_steps, batch_size, lr, 650)

plt.figure()
plt.plot(test_x, f_true(test_x), label="truth")
plt.scatter(evaluated_x, evaluated_pred, s=3, c='orange', label="predicted")
plt.legend()
plt.show()

**Question: Describe the performance of this model.** Please include the answer in your submission of the written assignment.

In [ ]:
m, singular_values, principal_feature = svd_local_linearization(model)
visualize(m, singular_values, principal_feature)

**Question: Describe your observation and compare it with the visualization of the local linearization of neural network with a single hidden layer.** Please include the figures and your answer in your submission of the written assignment.

## Out-of-Distribution Generalization

In this section, we investigate the impact of mismatched data distributions on the performance of the neural network by generating a new training set with $x$ values sampled from the interval -1.0 to 0.4.

In [ ]:
# Generate training data from -1 to 0.4
np.random.seed(50)
ood_train_x = np.random.uniform(low=-1.0, high=0.4, size=TRAIN_N)
ood_train_y = f_true(ood_train_x) + np.random.randn(TRAIN_N) * TRAIN_SIGMA

plt.figure()
plt.scatter(ood_train_x, ood_train_y, s=3)
plt.xlim(-1.05, 1.05)
plt.show()

In [ ]:
np.random.seed(600)
torch.manual_seed(600)
model.apply(better_init)

In [ ]:
lr = 0.1
batch_size = 32
n_steps = int(TRAIN_N / batch_size * 10)

model, evaluated_x, evaluated_pred = train_model(model, n_steps, batch_size, lr, 650, ood_train_x, ood_train_y)

plt.figure()
plt.plot(test_x, f_true(test_x), label="truth")
plt.scatter(evaluated_x, evaluated_pred, s=3, c='orange', label="predicted")
plt.legend()
plt.show()

**Question: Describe the performance of this model.** Please include the answer in your submission of the written assignment.

In [ ]:
m, singular_values, principal_feature = svd_local_linearization(model)
visualize(m, singular_values, principal_feature)

**Question: Describe your observation and compare it with (a) the visualization of the local linearization of the neural network with a single hidden layer; (b) the visualization of the local linearization of the properly-initialized neural network with 4 hidden layers.** Please include the figures and your answer in your submission of the written assignment.